In [1]:
import pandas as pd
import sys
sys.path.append('../src')
import pprint
pretty = pprint.PrettyPrinter(indent=4)
from cellopype import Cell, Pype

In [13]:
pp = Pype()

pp.cell_a = Cell(recalc=lambda: [1,2,3])
pp.cell_b = Cell(recalc=lambda: [4,5,6])

# note that the reference to the source cell instances includes the pype container:
# (allowing references to cells in other pypes)
pp.cell_c = Cell(recalc=lambda a, b: a+b, sources=[pp.cell_a, pp.cell_b])

In [14]:
pp.keys()

dict_keys(['cell_a', 'cell_b', 'cell_c'])

In [18]:
pp.cell_a

In [19]:
pp.cell_c.value

[1, 2, 3, 4, 5, 6]

In [23]:
# The cell name is also added back to the cell:
pp.cell_a.name   # handy for debugging if a cell can identity itself back to you

'cell_a'

In [24]:
pp.keys()

dict_keys(['cell_a', 'cell_b', 'cell_c'])

In [25]:
pp.cells

{'cell_a': <cellopype.cell.Cell at 0x7fefc0032520>,
 'cell_b': <cellopype.cell.Cell at 0x7fefc00323d0>,
 'cell_c': <cellopype.cell.Cell at 0x7fefc0032370>}

In [26]:
pp.recalc_all()

In [32]:
# recalculate and dump all cell values to a list of dicts 
pype_dict_list = pp.dump_values()

In [33]:
pretty.pprint(pype_dict_list)

[   {'name': 'cell_a', 'value': [1, 2, 3]},
    {'name': 'cell_b', 'value': [4, 5, 6]},
    {'name': 'cell_c', 'value': [1, 2, 3, 4, 5, 6]}]


In [29]:
# modify the local cell value in pp
pp.cell_a.value=[9,9,9]

In [30]:
# show the local cell value in pp has changed
pp.cell_a.value

[9, 9, 9]

In [35]:
# restore values from the dump
pp.load_values(pype_dict_list)
pp.cell_a.value

[1, 2, 3]

In [14]:
# if you want to dump & restore the entire pype, including recalc logic & handlers:
import dill  # not pickle, because lambdas

with open('pype_p.dill', 'wb') as file:
    dill.dump(pp, file)
with open('pype_p.dill', 'rb') as inp:
    pp2 = dill.load(inp)

In [16]:
# the pp2 pype is now a full & independent copy of pp
pp2.c.sources

In [18]:
# test recalc_handlers to verify those have been restored
pp2.c.recalc_handler(pp2.a.value, pp2.b.value)

,value
0,11
1,22
2,33
